# Fire response traits in plants from field samples

This script contains  R code to query tables in the database and download detailed plant species records for each quadrat.

## Load libraries

In [1]:
library(RPostgreSQL)
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)
library(lubridate)

Loading required package: DBI


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Connect to the database

Read database credentials

In [2]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")[-1]
  tmp <- strsplit(tmp,'=',fixed=2)
  dbinfo <- unlist(lapply(tmp,function(x) x[2]))
  names(dbinfo) <- unlist(lapply(tmp,function(x) x[1]))
  dbinfo <- data.frame(t(dbinfo),stringsAsFactors=F)
   rm(tmp)
} else {
  cat("No database information found")
}

Connection to the postgresql server (remember to update .pgpass file)

In [3]:
drv <- dbDriver("PostgreSQL") ## 
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

In [4]:
qry <- 
'WITH FH AS (SELECT visit_id, visit_date, MAX(earliest_date) as min_fire_date, MAX(latest_date) as max_fire_date
FROM form.field_visit
LEFT JOIN form.fire_history
ON visit_id=site_label
GROUP BY visit_id, visit_date)
SELECT visit_id, visit_date, sample_nr,
sample_method, quadrat_area,
min_fire_date, max_fire_date,
species, species_code,
resprout_organ, seedbank,
adults_unburnt,
resprouts_live, resprouts_died, resprouts_kill, resprouts_reproductive,
recruits_live, recruits_died, recruits_reproductive,
scorch, life_stage
FROM form.quadrat_samples
LEFT JOIN form.field_samples 
 USING (visit_id,visit_date,sample_nr)
LEFT JOIN FH
 USING (visit_id,visit_date)
'

In [5]:
quadrat.samples <- dbGetQuery(con, qry)


In [6]:
head(quadrat.samples)

,visit_id,visit_date,sample_nr,sample_method,quadrat_area,min_fire_date,max_fire_date,species,species_code,resprout_organ,⋯,adults_unburnt,resprouts_live,resprouts_died,resprouts_kill,resprouts_reproductive,recruits_live,recruits_died,recruits_reproductive,scorch,life_stage
,<chr>,<date>,<int>,<chr>,<dbl>,<date>,<date>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Goodenia ovata,3192,NA,⋯,0,0,NA,NA,0,100,NA,0,NA,NA
2,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Gonocarpus tetragynus,3247,NA,⋯,0,100,NA,NA,0,0,NA,0,NA,NA
3,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Eucalyptus blaxlandii,4058,NA,⋯,0,0,NA,NA,0,16,NA,0,NA,NA
4,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Eucalyptus viminalis,4197,NA,⋯,0,0,NA,NA,0,1,NA,0,NA,NA
5,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Pteridium esculentum,6403,NA,⋯,0,32,NA,NA,0,0,NA,0,NA,NA
6,CRC09B7UVH,2021-02-03,1,NA,NA,2020-02-10,2020-02-10,Opercularia aspera,5697,None,⋯,0,0,NA,NA,0,4,NA,0,NA,NA


In [8]:
quadrat.samples %>% 
    group_by(sample_year=year(visit_date) >2018) %>%
    summarise(
        locations = n_distinct(visit_id),
        visits  = n_distinct(visit_id, visit_date),
              samples  = n_distinct(visit_id, visit_date, sample_nr),
              records = n(), 
              species = n_distinct(species), 
              codes = n_distinct(species_code)) %>% 
    arrange(sample_year)

sample_year,locations,visits,samples,records,species,codes
<lgl>,<int>,<int>,<int>,<int>,<int>,<int>
FALSE,61,85,510,8588,181,149
TRUE,92,93,760,10654,933,906


We save the downloaded data to a RDS file. This will be uploaded to OSF cloud storage, this file will not be tracked by git.

In [8]:
saveRDS(file='../data/Quadrat-sample-data.rds',quadrat.samples)

In [9]:
dbDisconnect(con)

[1] TRUE